In [1]:

import os

os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

from src.data.dataset import encode_self_supervised_dataset
from src.models.tokenizer import load_tokenizer
from transformers import (
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments,
)


c:\Users\Auguste Verdier\Desktop\TeleoClassification



In [2]:
#import DNABERT2 model
from mlm_pre_training.dnabert2 import bert_layers
from transformers import TrainingArguments, AutoModel, EarlyStoppingCallback

In [3]:
#tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
tokenizer = load_tokenizer(4)

c:\ProgramData\anaconda3\envs\torch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm_probability= 0.20,)

In [5]:
train_dataset, val_dataset= encode_self_supervised_dataset(tokenizer,dir_path='mlm_pre_training/data/teleo/fold1/')


In [6]:
model = bert_layers.BertForMaskedLM.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
c:\Users\Auguste Verdier\Desktop\TeleoClassification\mlm_pre_training\dnabert2\bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


In [7]:
train_arg = TrainingArguments(
    output_dir = "mlm_pre_training/output/teleo_2/fold_1/checkpoint",
    eval_strategy = "steps",               # Directory for saved model
    logging_steps= 800,
    save_steps= 800,
    save_strategy="steps",                 # Save checkpoint at the end of every epoch
    save_total_limit=2,                    # Keep only the last and the best checkpoint
    load_best_model_at_end=True,           # Load the best model found at the end of training
    metric_for_best_model="eval_loss",     # Metric to use to determine the best model
    greater_is_better=False,       
    learning_rate=2e-5,
    num_train_epochs=50,
    weight_decay=0.01,
    logging_dir='mlm_pre_training/output/teleo_2/fold_1/logs')


trainer = Trainer(
    model=model,
    args=train_arg,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=20)]
)

In [ ]:
trainer.train()

In [9]:
eval_results = trainer.evaluate()
print(f"Evaluation Loss: {eval_results['eval_loss']}")

  0%|          | 0/189 [00:00<?, ?it/s]

{'eval_loss': 2.130812644958496, 'eval_runtime': 3.4089, 'eval_samples_per_second': 442.081, 'eval_steps_per_second': 55.443, 'epoch': 30.69}
Evaluation Loss: 2.130812644958496


In [12]:
import torch

In [21]:
trainer.save_model("mlm_pre_training/output/teleo_2/fold_1/best_model")
best_model = torch.load("mlm_pre_training/output/teleo_2/fold_1/checkpoint/checkpoint-1144800/pytorch_model.bin")
model.load_state_dict(best_model)

C:\Users\Auguste Verdier\AppData\Local\Temp\ipykernel_18824\1917967198.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load("mlm_pre_training/output/t

<All keys matched successfully>

In [22]:
model.eval()
trainer.model = model
validation_results = trainer.evaluate(eval_dataset=val_dataset)
print(f"Validation Loss: {validation_results['eval_loss']}")

  0%|          | 0/189 [00:00<?, ?it/s]

{'eval_loss': 2.114042282104492, 'eval_runtime': 4.2122, 'eval_samples_per_second': 357.774, 'eval_steps_per_second': 44.87, 'epoch': 30.69}
Validation Loss: 2.114042282104492


In [20]:
dna_sequences = ["ATCGT", "CGTAA", "TGCAT"]
tokenized_sequences = [tokenizer.tokenize(seq) for seq in dna_sequences]

batch = data_collator(tokenized_sequences)
print(batch)

ValueError: too many dimensions 'str'